# Exercise 1
## Character-level recurrent sequence-to-sequence model
### By: Daniel Mehta

---

## Inports

In [1]:
import numpy as np
import os
from pathlib import Path
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from tqdm import tqdm

---

## Dataset Path Setup

In [2]:
# setting path to dataset
data_dir = Path("fra-eng")
data_path = data_dir/"fra.txt"

In [3]:
if not data_path.exists():
    raise FileNotFoundError(f"Dataset not found at {data_path}")

print(f"Dataset located at: {data_path}")

Dataset located at: fra-eng\fra.txt


---

## Data Exploration and Cleaning

In [4]:
# Reading the file and split into lines
with open(data_path,"r",encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

In [5]:
print(f"Total sentence pairs in file: {len(lines)}")
print("Sample lines:")
for i in range(5):
    print(lines[i])

Total sentence pairs in file: 237838
Sample lines:
Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)


In [6]:
# Separating into English and French
pairs =[line.split("\t") for line in lines]
english_sentences =[pair[0] for pair in pairs]
french_sentences =[pair[1] for pair in pairs]

In [7]:
print("\nExample pair:")
print("EN:",english_sentences[0])
print("FR:",french_sentences[0])


Example pair:
EN: Go.
FR: Va !


---

## Configuration

In [8]:
# setting up seed
SEED = 5501
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [9]:
# Settubg yo start and end tokens
START_TOKEN="\t"
END_TOKEN="\n"

In [10]:
#  hyperparameters
batch_size = 64 # Batch size
epochs =100 # epochs of training
latent_dim = 256 #Latent dimensionality of the encoding space
num_samples = 10000  # Num of samples

print(f"batch_size={batch_size}, epochs={epochs}, latent_dim={latent_dim}, num_samples={num_samples}")
print(f"Decoder tokens -> start: {repr(START_TOKEN)}, end: {repr(END_TOKEN)}")

batch_size=64, epochs=100, latent_dim=256, num_samples=10000
Decoder tokens -> start: '\t', end: '\n'


In [11]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("PyTorch version:",torch.__version__)
print("CUDA available:",torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PyTorch version: 2.7.1+cu118
CUDA available: True
GPU: NVIDIA GeForce RTX 4060


In [12]:
# Building vocabularies

#  sorted unique characters for each language
input_characters = sorted(list(set("".join(english_sentences))))
target_characters = sorted(list(set("".join(french_sentences))))

#mapping dicts
input_char_to_idx ={char: idx for idx, char in enumerate(input_characters)}
input_idx_to_char ={idx: char for char, idx in input_char_to_idx.items()}

target_char_to_idx ={char: idx for idx, char in enumerate(target_characters)}
target_idx_to_char ={idx: char for char, idx in target_char_to_idx.items()}

# vocabulary sizes
input_vocab_size = len(input_characters)
target_vocab_size = len(target_characters)

print(f"Input vocab size: {input_vocab_size}")
print(f"Target vocab size: {target_vocab_size}")

Input vocab size: 90
Target vocab size: 113


In [13]:
# Add start/end tokens to french
french_sentences = [START_TOKEN+s+END_TOKEN for s in french_sentences]

# Limit samples
english_sentences = english_sentences[:num_samples]
french_sentences = french_sentences[:num_samples]

In [14]:
# Getting the sorted unique characters
input_characters = sorted(list(set("".join(english_sentences))))
target_characters = sorted(list(set("".join(french_sentences))))

In [15]:
# Map char to index
input_char_to_idx = {char: idx for idx,char in enumerate(input_characters)}
input_idx_to_char = {idx: char for char,idx in input_char_to_idx.items()}

target_char_to_idx = {char: idx for idx,char in enumerate(target_characters)}
target_idx_to_char = {idx: char for char,idx in target_char_to_idx.items()}

In [16]:
# Vocabulary sizes
input_vocab_size =len(input_characters)
target_vocab_size =len(target_characters)

print(f"Input vocab size: {input_vocab_size}")
print(f"Target vocab size: {target_vocab_size}")

Input vocab size: 70
Target vocab size: 91


In [17]:
# Convertting to index tensors
src_tensors = [torch.tensor([input_char_to_idx[ch] for ch in s],dtype=torch.long) 
               for s in english_sentences]
tgt_input_tensors = [torch.tensor([target_char_to_idx[ch] for ch in s[:-1]],dtype=torch.long) 
                     for s in french_sentences]
tgt_target_tensors = [torch.tensor([target_char_to_idx[ch] for ch in s[1:]],dtype=torch.long) 
                      for s in french_sentences]

# Pad sequences, 0 will be pad index
src_tensors=pad_sequence(src_tensors, batch_first=True,padding_value=0)
tgt_input_tensors=pad_sequence(tgt_input_tensors, batch_first=True,padding_value=0)
tgt_target_tensors=pad_sequence(tgt_target_tensors, batch_first=True,padding_value=0)

# Create Dataloader
dataset = list(zip(src_tensors,tgt_input_tensors,tgt_target_tensors))
train_dataloader =DataLoader(dataset,batch_size=batch_size,shuffle=True)

print(f"Batches in train_dataloader: {len(train_dataloader)}")

Batches in train_dataloader: 157


---

## Building the model

In [18]:
embed_dim = 128 # it must be smaller or equal to the latent dim

class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embed_dim,latent_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_vocab_size, embed_dim)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=latent_dim,
            num_layers=1,
            batch_first=True
        )


    def forward(self, src_idxs):
        # src_idxs:(batch, src_len)
        embedded =self.embedding(src_idxs) # (batch, src_len, embed_dim)
        outputs,(h,c) =self.lstm(embedded) #outputs not used,keep states
        return h,c

class Decoder(nn.Module):
    def __init__(self, target_vocab_size, embed_dim,latent_dim):
        super().__init__()
        self.embedding =nn.Embedding(target_vocab_size,embed_dim)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=latent_dim,
            num_layers=1,
            batch_first=True
        )
        self.fc_out =nn.Linear(latent_dim,target_vocab_size)
        
    def forward(self, tgt_idxs, hidden, cell):
        # tgt_idxs:(batch, tgt_len) with teacher forcing
        embedded = self.embedding(tgt_idxs)# (batch, tgt_len,embed_dim)
        outputs, (h,c) = self.lstm(embedded, (hidden,cell))
        logits = self.fc_out(outputs)#(batch, tgt_len, target_vocab_size)
        return logits, h,c


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder =encoder
        self.decoder =decoder

    def forward(self, src_idxs, tgt_input_idxs):
        # training forward pass with teacher forcing
        h,c =self.encoder(src_idxs)
        logits,_,_=self.decoder(tgt_input_idxs, h,c)
        return logits


#Instantiate and move to device
encoder = Encoder(input_vocab_size, embed_dim, latent_dim)
decoder = Decoder(target_vocab_size, embed_dim, latent_dim)
model =Seq2Seq(encoder, decoder).to(device)

# Loss and optimizer
PAD_IDX =None
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX) if PAD_IDX is not None else nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(70, 128)
    (lstm): LSTM(128, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(91, 128)
    (lstm): LSTM(128, 256, batch_first=True)
    (fc_out): Linear(in_features=256, out_features=91, bias=True)
  )
)


---

## Train model

In [19]:
for epoch in range(1, epochs+ 1):
    model.train()
    total_loss =0

    for src, tgt_input, tgt_target in tqdm(train_dataloader,desc=f"Epoch {epoch}/{epochs}"):
        src = src.to(device)
        tgt_input =tgt_input.to(device)
        tgt_target =tgt_target.to(device)

        optimizer.zero_grad()

        # Forward pass
        output_logits = model(src, tgt_input) #(batch, tgt_len, vocab_size)

        # Reshape for loss, mergeing batch & time dims
        output_logits = output_logits.reshape(-1,target_vocab_size)
        tgt_target = tgt_target.reshape(-1)

        # Compute loss
        loss = criterion(output_logits,tgt_target)
        loss.backward()

        optimizer.step()

        total_loss+=loss.item()

    avg_loss = total_loss/len(train_dataloader)
    print(f"Epoch {epoch} | Loss: {avg_loss:.4f}")

# Save model
torch.save(model.state_dict(),"seq2seq_model.pth")
print("Model saved to seq2seq_model.pth")

Epoch 1/100: 100%|██████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 171.16it/s]


Epoch 1 | Loss: 0.8973


Epoch 2/100: 100%|██████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 192.48it/s]


Epoch 2 | Loss: 0.5402


Epoch 3/100: 100%|██████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 200.36it/s]


...

Epoch 98 | Loss: 0.0301


Epoch 99/100: 100%|█████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 205.43it/s]


Epoch 99 | Loss: 0.0309


Epoch 100/100: 100%|████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 205.92it/s]

Epoch 100 | Loss: 0.0303
Model saved to seq2seq_model.pth


---

## Run inference (sampling)

In [20]:
def decode_sequence(model, src_seq, max_target_len=100):
    model.eval()
    
    # encode the input sequence to get initial hidden and cell state
    with torch.no_grad():
        h,c=model.encoder(src_seq)

        # Start token as the first decoder input
        start_idx=target_char_to_idx[START_TOKEN]
        decoder_input=torch.tensor([[start_idx]],dtype=torch.long,device=device)

        decoded_chars=[]

        for _ in range(max_target_len):
            # Pass throughthe  decoder for 1 step
            output,h,c =model.decoder(decoder_input,h,c)

            # Output is (batch=1, seq_len=1, vocab_size) to take last step
            output_char_idx=output.argmax(2).item()
            sampled_char=target_idx_to_char[output_char_idx]

            # Stop if end token
            if sampled_char ==END_TOKEN:
                break

            decoded_chars.append(sampled_char)

            # Use predicted char as next input
            decoder_input = torch.tensor([[output_char_idx]],dtype=torch.long,device=device)

    return "".join(decoded_chars)

In [21]:
def prepare_input_sentence(sentence):
    #Converting string to tensor of indices and add batch dimension
    idxs = [input_char_to_idx[ch] for ch in sentence]
    tensor = torch.tensor(idxs, dtype=torch.long).unsqueeze(0).to(device)
    return tensor

In [22]:
for _ in range(20):
    idx = random.randint(0, len(english_sentences) - 1)
    test_sentence = english_sentences[idx]
    input_tensor = prepare_input_sentence(test_sentence)
    translation = decode_sequence(model, input_tensor)

    print(f"EN: {test_sentence}")
    print(f"PREDICTED: {translation}")
    print(f"REFERENCE: {french_sentences[idx]}")
    print("-" * 40)

EN: I admire you.
PREDICTED: Je t'admire.
REFERENCE: 	Je vous admire.

----------------------------------------
EN: Many thanks.
PREDICTED: Merci mille fois !
REFERENCE: 	Mille mercis !

----------------------------------------
EN: Are you ready?
PREDICTED: Es-tu prêt ?
REFERENCE: 	Est-ce que vous êtes prêts ?

----------------------------------------
EN: Shame on you.
PREDICTED: Honte à toi.
REFERENCE: 	Honte à toi.

----------------------------------------
EN: They lied.
PREDICTED: Ils se sont en avance.
REFERENCE: 	Ils ont menti.

----------------------------------------
EN: We yawned.
PREDICTED: On a eu de la folle.
REFERENCE: 	Nous avons bâillé.

----------------------------------------
EN: Relax.
PREDICTED: lenu un piège.
REFERENCE: 	Détends-toi !

----------------------------------------
EN: I'm a girl.
PREDICTED: Je suis paresseux.
REFERENCE: 	Je suis une fille.

----------------------------------------
EN: Tom is OK now.
PREDICTED: Thomas est OK maintenant.
REFERENCE: 	Thomas 